# It's a Machine and Natural Language Model

In [1]:
from src.IaMaN.base import LM
from src.utils.data import load_wikitext
from src.utils.munge import stick_spaces
from collections import defaultdict
from collections import Counter
from tqdm import tqdm
import numpy as np
import os, re

seed = 691; max_char = 200_000_000
m = 10; space = True; fine_tune = False; num_posttrain = 100; noise = 0.001
positional = 'dependent'; positionally_encode = True; bits = 50; btype = 'nf'; 
ms_init = 'waiting_time'; update = {'representation', 'contexts', 'tokenizer'} # , 'targets'
runners = 10; gpu = False; tokenizer = 'hr-bpe' # 'sentokenizer' #

total_posttrain = 0; ptdocs = []
if num_posttrain:
    num_posttrain = 0 if num_posttrain == 'all' else num_posttrain
    print("Loading WikiText-103 as post-training data...")
    posttrain_docs = load_wikitext(v = '103', split = "train", num_articles = num_posttrain, seed = seed, space = space, rebuild = False)
    ptdocs = [["".join(s) for s in d['document']] for d in posttrain_docs]; ptdocs = [d for d in ptdocs if len(d)]
print("Loading WikiText-2 LM'ing data...")
train_docs = load_wikitext(v = '2', split = "train", num_articles = 0, seed = seed, space = space, rebuild = True)
test_docs = load_wikitext(v = '2', split = "test", num_articles = 0, seed = seed, space = space, rebuild = True)
test_docs = [doc for doc in test_docs if len(doc['text']) and (len(doc['text']) <= max_char)][:6]
train_docs = [doc for doc in train_docs if len(doc['text']) and (len(doc['text']) <= max_char)][:60]
print('Avail. post-train, total post-train, Avail. gold, total gold-train, total test-gold: ', 
      total_posttrain, len(ptdocs), len(train_docs) + len(test_docs), len(train_docs), len(test_docs))

Loading WikiText-103 as post-training data...
Loading WikiText-2 LM'ing data...
Avail. post-train, total post-train, Avail. gold, total gold-train, total test-gold:  0 99 66 60 6


In [2]:
docs = [["".join(s) for s in d['document']] for d in train_docs]
tdocs = [["".join(s) for s in d['document']] for d in test_docs]
covering = [[re.split('( )', ''.join(s)) for s in d['document']] for d in train_docs]
tcovering = [[re.split('( )', ''.join(s)) for s in d['document']] for d in test_docs]
if not space:
    for d_i, d in enumerate(covering):
        for s_i, s in enumerate(d):
            covering[d_i][s_i] = stick_spaces(s)
    for d_i, d in enumerate(tcovering):
        for s_i, s in enumerate(d):
            tcovering[d_i][s_i] = stick_spaces(s)            
covering_vocab = set([t for d in covering for s in d for t in s])
model = LM(m = m, tokenizer = tokenizer, noise = noise, seed = seed, space = space, positional = positional,
           positionally_encode = positionally_encode, bits = bits, runners = runners, gpu = gpu, 
           btype = btype, ms_init = ms_init)
docs_name = f'WT2-{len(train_docs)}'
ptdocs_name = f'WT103-{len(ptdocs)}'
data_streams = model.fit(docs, docs_name, update = update, fine_tune = fine_tune, covering = covering)

Training tokenizer


Fitting:   8%|▊         | 8/100 [00:24<04:37,  3.02s/it]


Built a vocabulary of 15533 types
Pre-processing data...


Counting tag-tag transition frequencies: 100%|██████████| 60/60 [00:00<00:00, 109.84it/s]


Counting documents and aggregating counts...


Building dense decoders: 100%|██████████| 15/15 [00:00<00:00, 21.19it/s]
Building transition matrices for tag-sequence decoding:   0%|          | 0/6 [00:00<?, ?it/s]/code/IaMaN/src/IaMaN/base.py:625: RuntimeWarning: invalid value encountered in true_divide
  self._trXs[ltype] = (lambda M: M/M.sum(axis = 1)[:,None])(self._trXs[ltype])
Building transition matrices for tag-sequence decoding: 100%|██████████| 6/6 [00:00<00:00, 9931.26it/s]

Model params, types, encoding size, contexts, vec dim, max sent, and % capacity used: 1191420 11488 50 1071 63 1211 15.643


In [4]:
model._ms

{'form': 6,
 'bits': 5,
 'nov': 2,
 'iat': 2,
 'bot': 2,
 'eot': 2,
 'eos': 2,
 'eod': 2}

In [5]:
model.generate(m = 50, seed = seed)

EncouragedEncouragedEncouraged1795 @-@ , but Szent into in 15 ringmainsinto   e from Wrapped n from off 20 also design with 


In [6]:
out = model.stencil(docs = tdocs, return_output = True, covering = tcovering)

Pre-processing data...


Evaluating LM:  17%|█▋        | 1/6 [00:03<00:19,  3.94s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 17.0 45.37 532.26 1697 983


Evaluating LM:  33%|███▎      | 2/6 [00:27<01:03, 15.75s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 33.0 54.31 737.39 10309 5853


...

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  4 67.0 45.95 530.3 15746 9075


Evaluating LM:  83%|████████▎ | 5/6 [01:24<00:17, 17.73s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  5 83.0 49.16 695.23 4049 2292


Evaluating LM: 100%|██████████| 6/6 [01:35<00:00, 15.91s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  6 100.0 55.23 755.67 4867 2792


In [7]:
print('micro: ', round(1/(10**(np.log10([p[0] for ps in out for p in ps]).mean())), 3),
      '; macro: ', round(np.mean([1/(10**(np.log10([p[0] for p in ps]).mean())) for ps in out]), 3),
      '; micro (w/out space): ', round(1/(10**(np.log10([p[1] for ps in out for p in ps if p[1] is not None]).mean())), 3),
      '; macro (w/out space): ', round(np.mean([1/(10**(np.log10([p[1] for p in ps if p[1] is not None]).mean())) for ps in out]), 3))
# P1: micro:  28.47 ; macro:  28.98 ; micro (w/out space):  265.351 ; macro (w/out space):  274.253
#* P2: micro:  25.921 ; macro:  26.308 ; micro (w/out space):  243.428 ; macro (w/out space):  248.857

micro:  50.434 ; macro:  51.156 ; micro (w/out space):  648.166 ; macro (w/out space):  682.141


In [8]:
_ = model.fit(ptdocs, ptdocs_name, update = update, fine_tune = fine_tune, bits = bits)
if ptdocs:
    model.generate(m = 50, seed = seed)
if fine_tune and ptdocs:
    model.fine_tune(docs, covering = covering, streams = data_streams)

Training tokenizer


Fitting:   7%|▋         | 7/100 [00:56<12:26,  8.03s/it]


Built a vocabulary of 28191 types
Pre-processing data...


Counting tag-tag transition frequencies: 100%|██████████| 159/159 [00:00<00:00, 159.17it/s]


Counting documents and aggregating counts...


Building dense decoders: 100%|██████████| 15/15 [00:00<00:00, 24.87it/s]
Building transition matrices for tag-sequence decoding: 100%|██████████| 6/6 [00:00<00:00, 12470.68it/s]


Model params, types, encoding size, contexts, vec dim, max sent, and % capacity used: 1876297 11080 50 1071 63 1248 25.542
BornHadjiBornorigins <unk> , which 1871 command d fence forepot 18 . . 
Theremedyadvent has to expand 10 000 . The hurricane 


In [9]:
if ptdocs:
    out = model.stencil(docs = tdocs, return_output = True, covering = tcovering)

Pre-processing data...


Evaluating LM:  17%|█▋        | 1/6 [00:03<00:18,  3.74s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 17.0 28.75 255.28 1696 982


Evaluating LM:  33%|███▎      | 2/6 [00:26<01:00, 15.20s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 33.0 36.12 346.53 10493 6037


...

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  4 67.0 31.1 266.7 15865 9194


Evaluating LM:  83%|████████▎ | 5/6 [01:20<00:17, 17.02s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  5 83.0 31.91 333.35 4057 2300


Evaluating LM: 100%|██████████| 6/6 [01:31<00:00, 15.28s/it]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  6 100.0 29.93 278.82 4909 2834


In [10]:
if ptdocs:
    print('micro: ', round(1/(10**(np.log10([p[0] for ps in out for p in ps]).mean())), 3),
          '; macro: ', round(np.mean([1/(10**(np.log10([p[0] for p in ps]).mean())) for ps in out]), 3),
          '; micro (w/out space): ', round(1/(10**(np.log10([p[1] for ps in out for p in ps if p[1] is not None]).mean())), 3),
          '; macro (w/out space): ', round(np.mean([1/(10**(np.log10([p[1] for p in ps if p[1] is not None]).mean())) for ps in out]), 3))

micro:  32.427 ; macro:  31.947 ; micro (w/out space):  300.933 ; macro (w/out space):  305.532
